# Homework 10: Cleaning data with Regular Expressions

Time to use regular expressions!

# Hints and notes

### Opening files in subdirectories

Notice that this notebook might be **homework/**, but!!! the csvs and text files might be in **homework/scraped/** or **/homework/scraped/minutes_pdfs** or **/homework/pdfs/**. To open a file in a subdirectory, instead of having the filename be `"file.csv"` you'll just use `"some/subfolder/file.csv"`

### Opening text files

This will open up a file, read it in and show you the first 500 characters.

```python
contents = open("your-filename.txt").read()
contents[0:500]
```

> You might need `open("your-filename.txt", encoding="utf8").read()`

### Using regex

For some dumb reason you need to put `r` in front of the string you use when you're talking about regex. Just plain `"(\d\d\d)"` will usually work, but *sometimes* it won't and you'll need `r"(\d\d\d)`. It's best to just use the `r` all of the time, if you can remember!

### Using `.str.extract`

When you use `.str.extract`, you're always going to **capture one thing** and save it to a new column. You need to wrap the things you're interested in with parenthesis `(` `)`.

```python
df['phone_number'] = df['old_column'].str.extract(r"My phone number is (\d\d\d-\d\d\d-\d\d\d\d)")
```

### Setting pandas options

Pandas has a lot of options, like how many columns or rows it will show you, or how many characters it will show in a column before it stops showing you anything. Here are a few useful ones:

* `display.max_cols`: Number of columns to show at once
* `display.max_rows`: Number of rows to show at once
* `display.max_colwidth`: Maximum number of characters displayed from a string

You can set them using `pd.set_option("display.max_rows", 1000)`, for example, to show 1000 rows at a time. You can find a lot more at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html

### Regular expressions reference

I personally think http://www.regular-expressions.info/ is a wonderful wonderful reference (and tutorial), even if it's ugly! But here's a quick reference for you:

* `\d` is a digit
* `\d*` is zero or more digits 
* `\d+` is one or more digits
* `.` matches anything for ONE character
* `.*` is "give me anything forever"
* `\s` is whitespace, a.k.a. spaces and tabs
* `\w` is a word character, which includes capital and lowercase letters, numbers and hyphens.
* You can put `*` after anything, so `\w*` would mean "as many word characters as you can find"
* `\b` is a word boundary (you'll need the `r""` thing for this one)
* `( )` is a "capture group" for saving something
* `\1` is used when doing find/replace to say "put the first captured group here" (note, it's a dollar sign instead of a backslash in some editors)
* `[ABCDE]` is a character class, which means "match one of these, I don't care which"
* dollar sign means "end of the line"
* caret ^ means "beginning of the line"
* `\.` means "no really seriously I mean a period not just anything"
* You can use `\` with anything else that would normally be a special character, too, not just periods. `(` or `[` or whatever.

### Cleaning up extracted columns

Sometimes you get `\n` (newlines) or spaces or `\t` (tabs) or stuff at the beginning or the end of your column. `.str.strip()` will usually take care of that, just attach it after your `.str.extract()`

After you extract something, it's still a string even though you look at it and know it's a number. Use `.astype(int)` to turn it into an integer (no decimal) or `.astype(float)` to turn it into a float (yes decimal)

### Writing regular expressions in general

Even if I'm using regex in pandas or Python, I like to test them in my text editor with "Find." The highlighting really helps me see if I'm matching things! I also like to think "what stays the same?" when designing patterns, write those parts first, then fill in the blanks with what I want to capture.

## Importing

There might be more, I just wanted to put this up here for the `pd.set_option` part. It allows you to see a lot of content in a single column of pandas, which will be important for some parts below.

In [1]:
import re
import pandas as pd
pd.set_option('display.max_colwidth', 500)

# Part 1: Using `.str.extract` to pull data from columns in pandas

## 1.1 H&M

Open up `hm.csv` from the `scraped` directory. I want **four new columns**:

1. `price_original`, the original price, one of the new price
2. `price_discounted`, the discounted price
3. `pct_discount`, the percent discount
4. `article_id`, the article id (from the url)

Save as **hm_cleaned.csv**.

**Note:** When you look at it, it... won't look right. I don't know why, pandas is weird. Look at the `price` column by itself using `df['price']` before you write your regex.

**Tip:** Remember that `$` is a special regex symbol! You might need to escape it.

**Tip:** When doing `.str.extract`, the whole match doesn't get captured, only what you put `()` around! Think about anchoring to different points of the string, or things in the string.

**Tip:** Not all prices have cents!

**Tip:** Your first instinct about how to compute the percent discount is probably wrong

In [2]:
contents = open("scraped/hm.csv").read()
contents[0:500]

'name,price,url\nWashed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N\nCandle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D\nGlittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A\nTextured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C\nStoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A\nSlub-weave Cushion Cover,$3.99 $9.99,http://www.h'

In [3]:
df = pd.read_csv("scraped/hm.csv")
df.head()

,name,price,url
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A


In [4]:
df.columns

Index(['name', 'price', 'url'], dtype='object')

In [5]:
df.price

0       $59.99 $129
1      $6.99 $17.99
2      $7.99 $17.99
3      $6.99 $12.99
4     $17.99 $24.99
5       $3.99 $9.99
6      $7.99 $17.99
7      $7.99 $17.99
8      $9.99 $17.99
9     $27.99 $34.99
10      $2.99 $5.99
11      $2.99 $5.99
12     $9.99 $12.99
13    $39.99 $49.99
14    $39.99 $49.99
15     $7.99 $17.99
16    $19.99 $29.99
17      $2.99 $5.99
18     $7.99 $17.99
19    $14.99 $24.99
20     $9.99 $24.99
21     $7.99 $12.99
22     $7.99 $17.99
23      $4.99 $9.99
24     $9.99 $12.99
25     $7.99 $17.99
26      $3.99 $6.99
27     $7.99 $17.99
28      $2.99 $6.99
29     $9.99 $12.99
30    $14.99 $34.99
31     $7.99 $17.99
32    $24.99 $49.99
33    $29.99 $79.99
34     $4.99 $12.99
35    $19.99 $24.99
36      $2.99 $5.99
37    $14.99 $34.99
38     $6.99 $17.99
39     $6.99 $17.99
40     $6.99 $17.99
41         $99 $199
42       $54.99 $99
43     $6.99 $17.99
44     $4.99 $12.99
45     $7.99 $17.99
46      $3.99 $9.99
47     $6.99 $17.99
48     $4.99 $12.99
49      $2.99 $5.99


In [6]:
df.price.str.extract("\$(\d*.\d\d$)").head(3)             #("(\d*\.\d*)")

,0
0,129
1,17.99
2,17.99


In [7]:
# check .astype(float)
df['price_original'] = df.price.str.extract("\$(\d*.\d\d$)")


In [8]:
df['price_original'] = df['price_original'].astype(float)
df['price_original'].head(3)

0    129.00
1     17.99
2     17.99
Name: price_original, dtype: float64

In [9]:
# for price discounted:
df.price.str.extract("^\$(\d*\.\d*)").head(3)

,0
0,59.99
1,6.99
2,7.99


In [10]:
# df.price.str.extract("(\d*.\d\d$)") 
df['price_discounted'] = df.price.str.extract("^\$(\d*\.\d*)")


In [11]:
df['price_discounted'] = df['price_discounted'].astype(float)

In [12]:
# for PERCENT discount (pct_discount): original price - discount price : original price x 100% devide the final price 
df.head(3)
#df.price.str.extract("(\d*.\d\d$)")

,name,price,url,price_original,price_discounted
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,129.00,59.99
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,17.99,6.99
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,17.99,7.99


In [13]:
df['price_original'] - df['price_discounted']

0     69.01
1     11.00
2     10.00
3      6.00
4      7.00
5      6.00
6     10.00
7     10.00
8      8.00
9      7.00
10     3.00
11     3.00
12     3.00
13    10.00
14    10.00
15    10.00
16    10.00
17     3.00
18    10.00
19    10.00
20    15.00
21     5.00
22    10.00
23     5.00
24     3.00
25    10.00
26     3.00
27    10.00
28     4.00
29     3.00
30    20.00
31    10.00
32    25.00
33    50.00
34     8.00
35     5.00
36     3.00
37    20.00
38    11.00
39    11.00
40    11.00
41      NaN
42      NaN
43    11.00
44     8.00
45    10.00
46     6.00
47    11.00
48     8.00
49     3.00
50     5.00
51     6.00
52     3.00
53    45.00
54    10.00
55    45.00
56     3.00
57    10.00
58     7.00
59     5.00
dtype: float64

In [14]:
difference = df['price_original'] - df['price_discounted']

In [15]:
df['pct_discount'] = round(difference/df['price_original'] * 100,1)

In [16]:
df.pct_discount

0     53.5
1     61.1
2     55.6
3     46.2
4     28.0
5     60.1
6     55.6
7     55.6
8     44.5
9     20.0
10    50.1
11    50.1
12    23.1
13    20.0
14    20.0
15    55.6
16    33.3
17    50.1
18    55.6
19    40.0
20    60.0
21    38.5
22    55.6
23    50.1
24    23.1
25    55.6
26    42.9
27    55.6
28    57.2
29    23.1
30    57.2
31    55.6
32    50.0
33    62.5
34    61.6
35    20.0
36    50.1
37    57.2
38    61.1
39    61.1
40    61.1
41     NaN
42     NaN
43    61.1
44    61.6
45    55.6
46    60.1
47    61.1
48    61.6
49    50.1
50    50.1
51    46.2
52    50.1
53    56.3
54    55.6
55    56.3
56    50.1
57    40.0
58    53.9
59    27.8
Name: pct_discount, dtype: float64

In [17]:
#df['article_id'] = df.price.str.extract("(^[h].*$)")
df['article_id'] = df.url.str.extract(r"=(\d\d\d\d\d-[A-Z])")

df['article_id'].head(3)

0    13472-N
1    35079-D
2    72462-A
Name: article_id, dtype: object

In [18]:
df

,name,price,url,price_original,price_discounted,pct_discount,article_id
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,129.00,59.99,53.5,13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,17.99,6.99,61.1,35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,17.99,7.99,55.6,72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,12.99,6.99,46.2,58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,24.99,17.99,28.0,74242-A
5,Slub-weave Cushion Cover,$3.99 $9.99,http://www.hm.com/us/product/70965?article=70965-D,9.99,3.99,60.1,70965-D
6,Braided Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/62818?article=62818-B,17.99,7.99,55.6,62818-B
7,Jacquard-weave Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/69163?article=69163-B,17.99,7.99,55.6,69163-B
8,Scented Candle in Glass Holder,$9.99 $17.99,http://www.hm.com/us/product/40910?article=40910-C,17.99,9.99,44.5,40910-C
9,2-pack Curtain Panels,$27.99 $34.99,http://www.hm.com/us/product/69699?article=69699-B,34.99,27.99,20.0,69699-B


## 1.2 Sci-Fi Authors

Open up `sci-fi.csv` to clean. Get rid of the `\n` on the title and and give me six new columns:

* `avg_rating`
* `rating_count`
* `total_score`
* `score_votes`
* `series` the series the book belongs to
* `series_no` the book in the series that it is

For series, I'm talking about e.g. `(The Hunger Games, #1)` is `series` "The Hunter Games" and `series_no` 1.

Save as **sci-fi_cleaned.csv**.

**Tip:** You don't need regex to clean the title - there's a special thing that removes whitespace from the beginning/end of strings

**Tip:** Remember that `(` and `)` are special characters

**BONUS:** When you make the `total_score` column, pay close attention to it. If you notice the problem, fix it.

**BONUS:** You don't need these columns to be numbers, but life would be better if they were. 

In [19]:
import re
import pandas as pd
pd.set_option('display.max_colwidth', 500)

In [20]:
contents = open("scraped/sci-fi.csv").read()
contents[0:500]

'full_rating,full_score,rank,title,url\n" 4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n              and\n292 people voted\n              \u2003\n              \n            ",1,"\nThe Handmaid\'s Tale\n",/book/show/38447.The_Handmaid_s_Tale\n" 4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n              and\n282 people voted\n              \u2003\n              \n            ",2,"\nThe Hunger Games (The Hunger Games, #1)\n",/book/show/2767052-the-hunger-games\n" 3.76 avg rating — 922,308 ratings","\nscore: 2'

In [21]:
df = pd.read_csv("scraped/sci-fi.csv")
df.head()

,full_rating,full_score,rank,title,url
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,\nThe Handmaid's Tale\n,/book/show/38447.The_Handmaid_s_Tale
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"\nThe Hunger Games (The Hunger Games, #1)\n",/book/show/2767052-the-hunger-games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"\nFrankenstein, or The Modern Prometheus\n",/book/show/18490.Frankenstein_or_The_Modern_Prometheus
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"\nA Wrinkle in Time (A Wrinkle in Time Quintet, #1)\n",/book/show/18131.A_Wrinkle_in_Time
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,\nThe Left Hand of Darkness\n,/book/show/18423.The_Left_Hand_of_Darkness


In [22]:
df.columns
df.full_score.head(3)

0    \nscore: 28,539,\n              and\n292 people voted\n               \n              \n            
1    \nscore: 27,566,\n              and\n282 people voted\n               \n              \n            
2    \nscore: 20,049,\n              and\n205 people voted\n               \n              \n            
Name: full_score, dtype: object

In [23]:
df.full_rating.head(3)

0       4.07 avg rating — 785,502 ratings
1     4.34 avg rating — 5,212,935 ratings
2       3.76 avg rating — 922,308 ratings
Name: full_rating, dtype: object

In [24]:
df.full_rating.str.extract(r"(\d\.\d\d )avg").head(3) 

,0
0,4.07
1,4.34
2,3.76


In [25]:
df.full_score.head(3)

0    \nscore: 28,539,\n              and\n292 people voted\n               \n              \n            
1    \nscore: 27,566,\n              and\n282 people voted\n               \n              \n            
2    \nscore: 20,049,\n              and\n205 people voted\n               \n              \n            
Name: full_score, dtype: object

In [26]:
df['avg_rating'] = df.full_rating.str.extract(r"(\d\.\d\d )avg")
df.full_score.head(3)

0    \nscore: 28,539,\n              and\n292 people voted\n               \n              \n            
1    \nscore: 27,566,\n              and\n282 people voted\n               \n              \n            
2    \nscore: 20,049,\n              and\n205 people voted\n               \n              \n            
Name: full_score, dtype: object

In [27]:
df.full_rating.str.extract(r"rating (.*)ratings").head(3) 
#df.full_score.head(3)

,0
0,"— 785,502"
1,"— 5,212,935"
2,"— 922,308"


In [28]:
df['rating_count'] = df.full_rating.str.extract(r"rating (.*)ratings")
df['full_score'].head(3)
#df.full_score.head(3)

0    \nscore: 28,539,\n              and\n292 people voted\n               \n              \n            
1    \nscore: 27,566,\n              and\n282 people voted\n               \n              \n            
2    \nscore: 20,049,\n              and\n205 people voted\n               \n              \n            
Name: full_score, dtype: object

In [29]:
# df = df.replace(r'\\n',' ', regex=True) 

#df['full_score'].replace(r'\n',' ', regex=True) 
#df.full_score.head(3)

In [30]:
df.full_score

0     \nscore: 28,539,\n              and\n292 people voted\n               \n              \n            
1     \nscore: 27,566,\n              and\n282 people voted\n               \n              \n            
2     \nscore: 20,049,\n              and\n205 people voted\n               \n              \n            
3     \nscore: 17,684,\n              and\n185 people voted\n               \n              \n            
4     \nscore: 16,070,\n              and\n165 people voted\n               \n              \n            
5     \nscore: 12,935,\n              and\n134 people voted\n               \n              \n            
6     \nscore: 12,261,\n              and\n128 people voted\n               \n              \n            
7     \nscore: 11,238,\n              and\n117 people voted\n               \n              \n            
8     \nscore: 10,246,\n              and\n107 people voted\n               \n              \n            
9      \nscore: 9,907,\n             

In [31]:
# df.total_score creation
df.full_score.str.extract(r"\nscore: (\d+\,\d*)\,") 
#df.full_score.head(3)

,0
0,"28,539"
1,"27,566"
2,"20,049"
3,"17,684"
4,"16,070"
5,"12,935"
6,"12,261"
7,"11,238"
8,"10,246"
9,"9,907"


In [32]:
df['total_score'] = df.full_score.str.extract(r"\nscore: (\d+\,\d*)\,")

In [33]:
df['total_score'].head(3)

0    28,539
1    27,566
2    20,049
Name: total_score, dtype: object

In [34]:
# df.score_votes creation
df.full_score.str.extract(r"and\n(\d+) people") 

,0
0,292
1,282
2,205
3,185
4,165
5,134
6,128
7,117
8,107
9,104


In [35]:
df['score_votes'] = df.full_score.str.extract(r"and\n(\d+) people")

In [36]:
df['score_votes'].head(3)

0    292
1    282
2    205
Name: score_votes, dtype: object

In [37]:
# df.series creation
df.title.str.extract(r"\n(\*)\n") 

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [38]:
# the above didn't work so I need to get rid of /n etc.
df.title = df.title.replace('\n','', regex=True)

In [39]:
df.title.head()

0                                  The Handmaid's Tale
1              The Hunger Games (The Hunger Games, #1)
2               Frankenstein, or The Modern Prometheus
3    A Wrinkle in Time (A Wrinkle in Time Quintet, #1)
4                            The Left Hand of Darkness
Name: title, dtype: object

In [40]:
# df.series creation
df.title.str.extract(r"\((.*)\,") 

,0
0,NaN
1,The Hunger Games
2,NaN
3,A Wrinkle in Time Quintet
4,NaN
5,Divergent
6,The Hunger Games
7,The Giver
8,NaN
9,NaN


In [41]:
df['series'] = df.title.str.extract(r"\((.*)\,")

In [42]:
df['series'].head()

0                          NaN
1             The Hunger Games
2                          NaN
3    A Wrinkle in Time Quintet
4                          NaN
Name: series, dtype: object

In [43]:
# df.series_no creation
df.title.str.extract(r"\(.*\#(.*)\)") 

,0
0,NaN
1,1
2,NaN
3,1
4,NaN
5,1
6,2
7,1
8,NaN
9,NaN


In [44]:
df['series_no'] = df.title.str.extract(r"\#(.*)\)")

In [45]:
df['series_no'].head()

0    NaN
1      1
2    NaN
3      1
4    NaN
Name: series_no, dtype: object

## 1.3 Where you're just doing one of my former students' projects

Once upon a time my student Stefan did a project that involved some lawyer stuff. Most of the content was in PDFs, though! I converted them to text files and put them into the `pdfs` folder, and gave you code below to open up each of them and save their contents into a dataframe.

What a nice dataframe! I want you to add the following columns to it:

* `lawyer_app`, the applicant's lawyer (pro se means that they did it themselves, that's fine)
* `lawyer_gov`, the government's lawyer
* `judge`, the name of the judge
* `access`, whether the clearance is granted or denied (although you might miss a few)

Save as **court_cleaned.csv**.

**Note:** You can look at the original PDFs, they're also included.

**Note:** This uses a fun utility called `glob`, which is mostly fun because you use it as `glob.glob`. It's used to find files that match a certain filename pattern.

**BONUS:** You'll be happy once you get the judge, but make sure it doesn't have any extra punctuation on it.

**BONUS:** You can for some words using `.str.contains("blah")` and save it into new columns. Maybe `has_debt`, `has_bankruptcy`, etc.

> It's okay if it isn't perfect. Converting PDF into data rarely is! Usually you get 90% of it done with computers, then send people to enter the other 10% by hand.

In [46]:
import glob
filenames = glob.glob("pdfs/*.txt")
contents = [open(filename, encoding="utf8").read() for filename in filenames]
df = pd.DataFrame({'filename': filenames, 'content': contents})

In [47]:
df.head()

,filename,content
0,pdfs/11-02438.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-02438 \n\nFor Government: Stephanie C. Hess, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\..."
1,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counse..."
2,pdfs/11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is ..."
3,pdfs/11-07728.h1.pdf.txt,"DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n------------------------ \n \n\n \n\nISCR Case No. 11-07728 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n\n \n \n\nAppearances \n\n___________ \n\n \nDecision \n\n___________ \n\nFor Government: Julie R. Mendez, Esq., Department Counsel \n\nFor Applicant: Mark S. Zaid, Esq. \n\n \n\nHARVEY, Mark, Administrative Judge: \n \..."
4,pdfs/11-08313.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n\nISCR Case No. 11-08313 \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n--------------- \n \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\nFor Government: Julie R. Mendez, Esquire, Department Counsel \n\nFo..."


Okay, now do the work and **make those new columns!**

In [48]:
# df.lawyer_app creation
df.content.str.extract(r"For Applicant:(.*)") 

,0
0,Pro se
1,"Mark S. Zaid, Esq."
2,Pro se
3,"Mark S. Zaid, Esq."
4,Pro se
5,Pro se
6,NaN
7,"Ryan C. Nerney, Esquire"
8,Pro se
9,Pro se


In [49]:
df['lawyer_app'] = df.content.str.extract(r"For Applicant:(.*)") 

In [50]:
df['lawyer_app'].head(3)

0                 Pro se 
1     Mark S. Zaid, Esq. 
2                 Pro se 
Name: lawyer_app, dtype: object

In [51]:
# df.lawyer_gov creation
df.content.str.extract(r"For Government:(.*)") 

,0
0,"Stephanie C. Hess, Esq., Department Counsel"
1,"Robert J. Kilmartin, Esq., Department Counsel"
2,"Richard Stevens, Esq., Department Counsel"
3,"Julie R. Mendez, Esq., Department Counsel"
4,"Julie R. Mendez, Esquire, Department Counsel"
5,"David F. Hayes, Esq., Department Counsel"
6,NaN
7,"Richard Stevens, Esquire, Department Counsel"
8,"Tara Karoian, Esq., Department Counsel"
9,"Eric Borgstrom, Esquire, Department Counsel"


In [52]:
df['lawyer_gov'] = df.content.str.extract(r"For Government:(.*)") 

In [53]:
# df.judge creation
df.content.str.extract(r"(.*)Administrative Judge:") 

,0
0,"COACHER, Robert E.,"
1,"LOUGHRAN, Edward W.,"
2,"DUFFY, James F.,"
3,"HARVEY, Mark,"
4,"MARSHALL, Jr., Arthur E.,"
5,"MENDEZ, Francisco,"
6,NaN
7,"GALES, Robert Robinson,"
8,"DUFFY, James F.,"
9,"HOGAN, Erin C.,"


In [54]:
df['judge'] = df.content.str.extract(r"(.*)Administrative Judge:")

In [55]:
# df.access creation
df.content.str.extract(r"information is(.*)")

,0
0,denied.
1,granted.
2,NaN
3,granted.
4,denied.
5,NaN
6,NaN
7,granted.
8,denied.
9,granted.


In [56]:
# IDEM alternative for better result 
df.content.str.extract(r" is (denied|granted)", re.IGNORECASE)

,0
0,denied
1,granted
2,granted
3,granted
4,denied
5,denied
6,NaN
7,granted
8,denied
9,granted


In [57]:
df['access'] = df.content.str.extract(r"information is(.*)")

# ===Books: Jane Austen===

In [82]:
import glob
filenames = glob.glob("pdfs/*.txt")
contents = [open(filename, encoding="utf8").read() for filename in filenames]
df = pd.DataFrame({'filename': filenames, 'content': contents})

In [106]:
df.head()

,filename,content
0,pdfs/11-02438.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-02438 \n\nFor Government: Stephanie C. Hess, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\..."
1,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counse..."
2,pdfs/11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is ..."
3,pdfs/11-07728.h1.pdf.txt,"DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n------------------------ \n \n\n \n\nISCR Case No. 11-07728 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n\n \n \n\nAppearances \n\n___________ \n\n \nDecision \n\n___________ \n\nFor Government: Julie R. Mendez, Esq., Department Counsel \n\nFor Applicant: Mark S. Zaid, Esq. \n\n \n\nHARVEY, Mark, Administrative Judge: \n \..."
4,pdfs/11-08313.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n\nISCR Case No. 11-08313 \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n--------------- \n \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\nFor Government: Julie R. Mendez, Esquire, Department Counsel \n\nFo..."


## Look at the first 500 or so characters of it 

In [85]:
contents = open("data/Austen_Pride.txt").read()
contents[0:500]

' Pride and Prejudice\nby Jane Austen\nChapter 1\nIt is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.\nHowever little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.\n"My dear Mr. Bennet," said his lady to him one day, "have you heard that Nethe'

 Use a regular expression to find every "he" or "she" in the book. There should be about 3000 of them.

**Tip:** Do you know about **word boundaries?** `\b` means "the beginning of end of a word."

**Tip:** You might also want to use `re.IGNORECASE`. Maybe you'll need to google it? 

**Tip:** Do NOT use `re.compile`

In [91]:
# "first "he" : " Use a regular expression to find every "he" or "she""
df.content.str.extract(r".*(he\b).*", re.IGNORECASE)

,0
0,he
1,he
2,he
3,he
4,he
5,he
6,he
7,he
8,he
9,he


In [92]:
#"she" :  Use a regular expression to find every "he" or "she"

df.content.str.extract(r".*(she\b).*", re.IGNORECASE)

,0
0,NaN
1,NaN
2,NaN
3,she
4,she
5,she
6,NaN
7,She
8,she
9,NaN


## Use a regular expression to find those same "he" or "she"s, but also match *the word after it*

The first four should be:

* he is
* he had
* she told
* he came

In [95]:
#for "she" + word after
df.content.str.extract(r".*(she\b \w*).*", re.IGNORECASE)

,0
0,NaN
1,NaN
2,NaN
3,she
4,she
5,she can
6,NaN
7,She refused
8,she has
9,NaN


In [96]:
# for "he" + word after
df.content.str.extract(r".*(he\b \w*).*", re.IGNORECASE)

,0
0,he matter
1,he matter
2,he matter
3,he matter
4,he matter
5,he matter
6,he malfunction
7,he matter
8,he matter
9,he matter


## Use capture groups to save the pronoun (he/she) as one match and the word as another

The first five should look like

```
[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed')]```

In [104]:
# for "he" + word after
df.content.str.extract(r".*(he\b) (\w*).*")

,0,1
0,he,matter
1,he,matter
2,he,matter
3,he,matter
4,he,matter
5,he,matter
6,he,malfunction
7,he,matter
8,he,matter
9,he,matter


## Save those matches into a dataframe

You can give the column names with `columns=['pronoun', 'verb']`

In [105]:
#df['access'] = df.content.str.extract(r"information is(.*)")
#columns=['pronoun', 'verb']

## How many times is each pronoun used?

## Oh, wait, clean that up.

Make it only 'he' and 'she' lowercase.

It should be about 1600 'she' and 1300 'he'

## What are the top 20 most common verbs?

## What are the top 20 most common verbs for 'he', and the top 20 most common for 'she'

**Tip:** Don't use groupby, just filter. If you want to know how, though, you can also look at "value counts for different categories" on [this page](http://jonathansoma.com/lede/foundations-2017/classes/more-pandas/class-notes/)

## Who cries more, men or women? Give me a percentage answer.

**Tip:** It's `cried`, because of, you know, how books are written

## How much more common is 'he' than 'she' in J.R.R. Tolkein's Fellowship of the Ring? How does that compare to Pride and Prejudice?

The book is in the same directory.